In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("24121001_SparkSQL_execution_plan").getOrCreate()

24/12/10 15:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/10 15:02:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/10 15:02:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
emp_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/emp.csv')

In [10]:
emp_df.show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|null|    20|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|null|    20|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
| 9292|  JACK|

In [11]:
dept_df = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('data/dept.csv')

In [12]:
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [17]:
emp_df.createOrReplaceTempView('emp_df_view')

In [18]:
dept_df.createOrReplaceTempView('dept_df_view')

In [29]:
# 뷰에서 조인
join_query = '''
select *
from emp_df_view join dept_df_view on emp_df_view.deptno = dept_df_view.deptno
'''

In [30]:
viewEmpDept = spark.sql(join_query)
viewEmpDept.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|null|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|null|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|null|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|null|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|A

In [37]:
viewEmpDept.createOrReplaceTempView('viewEmpDept')
# 부서위치가 'NEW YORK' 인 직원 목록

v_query = '''
select *
from viewEmpDept
where loc = 'NEW YORK'
'''

In [45]:
#case1
spark.sql(v_query).show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|    10|ACCOUNTING|NEW YORK|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|    10|ACCOUNTING|NEW YORK|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|    10|ACCOUNTING|NEW YORK|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+



In [42]:
#subquery

q2 = '''
select *
from emp_df_view
where deptno = (
    select deptno
    from dept_df_view
    where loc='NEW YORK'
)
'''

In [44]:
#case2
spark.sql(q2).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|null|    10|
| 7839|  KING|PRESIDENT|null|1981-11-17|5000|null|    10|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|null|    10|
+-----+------+---------+----+----------+----+----+------+



In [48]:
#case1 실행계획 join
spark.sql(v_query).explain()

== Physical Plan ==
*(2) BroadcastHashJoin [deptno#166], [deptno#232], Inner, BuildRight, false
:- *(2) Filter isnotnull(deptno#166)
:  +- FileScan csv [empno#159,ename#160,job#161,mgr#162,hiredate#163,sal#164,comm#165,deptno#166] Batched: false, DataFilters: [isnotnull(deptno#166)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab15/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [id=#546]
   +- *(1) Filter ((isnotnull(loc#234) AND (loc#234 = NEW YORK)) AND isnotnull(deptno#232))
      +- FileScan csv [deptno#232,dname#233,loc#234] Batched: false, DataFilters: [isnotnull(loc#234), (loc#234 = NEW YORK), isnotnull(deptno#232)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab15/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNu

In [49]:
#case2 실행계획 - 서브쿼리 / 훨씬 효과적으로 보임 필터로 좁히는 방식으로 진행
spark.sql(q2).explain()

== Physical Plan ==
*(1) Filter (isnotnull(deptno#166) AND (deptno#166 = Subquery scalar-subquery#778, [id=#565]))
:  +- Subquery scalar-subquery#778, [id=#565]
:     +- *(1) Project [deptno#232]
:        +- *(1) Filter (isnotnull(loc#234) AND (loc#234 = NEW YORK))
:           +- FileScan csv [deptno#232,loc#234] Batched: false, DataFilters: [isnotnull(loc#234), (loc#234 = NEW YORK)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab15/src/data/dept.csv], PartitionFilters: [], PushedFilters: [IsNotNull(loc), EqualTo(loc,NEW YORK)], ReadSchema: struct<deptno:int,loc:string>
+- FileScan csv [empno#159,ename#160,job#161,mgr#162,hiredate#163,sal#164,comm#165,deptno#166] Batched: false, DataFilters: [isnotnull(deptno#166)], Format: CSV, Location: InMemoryFileIndex[file:/home/lab15/src/data/emp.csv], PartitionFilters: [], PushedFilters: [IsNotNull(deptno)], ReadSchema: struct<empno:int,ename:string,job:string,mgr:int,hiredate:string,sal:int,comm:int,deptno:int>




In [50]:
spark.stop()